# Day 1: Map with Points

> Map with point data (e.g., individual locations, points of interest, clusters). Focus on effective symbolization and density visualization.

Download the data:

```
overturemaps download -f geoparquet --type=water -o water.parquet
```

In [18]:
import sedona.db
import os
from lonboard import Map, ScatterplotLayer, viz
import wkls

In [8]:
sd = sedona.db.connect()

In [9]:
df = sd.read_parquet(
    "/Users/matthewpowers/data/overture/water.parquet"
)

In [10]:
df.to_view("base_water", True)

In [11]:
la = wkls.us.ca.losangeles.wkt()

In [12]:
query = f"""
select ST_Centroid(geometry) as geometry from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{la}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
 and ST_GeometryType(geometry) = 'ST_Polygon'
"""

In [13]:
res = sd.sql(query)

In [14]:
res.count()

10899

In [15]:
layer = ScatterplotLayer(
    res,
    radius_min_pixels=1
)
m = Map(layer)

In [16]:
m

Map(custom_attribution='', layers=(ScatterplotLayer(radius_min_pixels=1.0, table=arro3.core.Table
+-----------…

In [19]:
viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

![map](../images/day01_mp_points_la_pools.png)

![map](../images/day01_mp_points_la_pools.png)